# Dialogue and Narrative Coursework - Subtask 1 - Data Analysis

In [ ]:
## imports
%%capture
!pip install datasets

from datasets import load_dataset, Dataset
import pandas as pd

In [ ]:
# Link Google Drive to get new dataset
from google.colab import drive
drive.mount('/gdrive')

# %cd drive/MyDrive/Colab\ Notebooks/Dialogue\ &\ Narrative
%cd ~
%cd /gdrive/My\ Drive
%cd Colab\ Notebooks/Dialogue\ &\ Narrative
print()

%ls .

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/root
/gdrive/My Drive
/gdrive/My Drive/Colab Notebooks/Dialogue & Narrative

 bert_matt_1/                                 predictions_ground_truth.json
 data_cache/                                  predictions_subtask1.json
 doc2dial_rc_train.csv                        test_doc2vec.model
 doc2dial_rc_val.csv                         'Week 1'/
'Doc2Dial_Task_1_BERT - Copy.ipynb'          'Week 2'/
'Doc2Dial - Task 1 - BERT.ipynb'             'Week 3'/
'Doc2Dial - Task 1 - Cosine.ipynb'           'Week 4'/
'Doc2Dial - Task 1 - GroundTruths.ipynb'     'Week 5'/
'Doc2Dial - Task 1 - New Datasets.ipynb'     'Week 7'/
'Doc2Dial - Task 1 - NewTrainDataset.ipynb'  'Week 8'/
 predictions_cosine2.json                    'Week 9'/


## Load the data

In [ ]:
## A function to load a specific doc2dial dataset

def load_doc2dial_dataset(name='dialogue_domain', split='train'):
  cache_dir = "./data_cache"

  return load_dataset(
      "doc2dial",
      name=name,
      split=split,
      ignore_verifications=True,
      cache_dir=cache_dir,
  )


## Understanding an example

In [ ]:
## Load documents 

documents = load_doc2dial_dataset(
              name="document_domain",
              split="train",
            )

Reusing dataset doc2dial (./data_cache/doc2dial/document_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


In [ ]:
documents[0]

{'doc_html_raw': '<main class="content" id="content" role="main">\n\n<section>\n\n<div>\n<h2>Benefits Planner: Survivors | Planning For Your Survivors</h2>\n</div>\n</section>\n\n<section>\n\n<div>\n\n<div>\n\n\n</div>\n\n\n\n<article>\n<section>\n<p>As you plan for the future, you\'ll want to think about what your family would need if you should die now. Social Security can help your family if you have earned enough Social Security credits through your work.</p>\n<p><a>You can earn up to four credits each year</a>. In 2019, for example, you earn one credit for each $1,360 of wages or <a>self-employment</a> income. When you have earned $5,440, you have earned your four credits for the year.</p>\n<p>The number of credits needed to provide benefits for your survivors depends on your age when you die. No one needs more than 40 credits (10 years of work) to be eligible for any Social Security benefit. But, the younger a person is, the fewer credits they must have for family members to rece

In [ ]:
## Load dialogue turns

dialogues = load_doc2dial_dataset(
              name="dialogue_domain",
              split="validation",
            )

Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


In [ ]:
dialogues[0]

{'dial_id': 'dea7174409afbfe0af0ace21e7f318ae',
 'doc_id': 'Top 5 DMV Mistakes and How to Avoid Them#3_0',
 'domain': 'dmv',
 'turns': [{'da': 'query_condition',
   'references': [{'label': 'precondition', 'sp_id': '23'}],
   'role': 'user',
   'turn_id': 1,
   'utterance': 'My insurance ended so what should i do'},
  {'da': 'respond_solution',
   'references': [{'label': 'solution', 'sp_id': '24'},
    {'label': 'solution', 'sp_id': '25'},
    {'label': 'solution', 'sp_id': '26'}],
   'role': 'agent',
   'turn_id': 2,
   'utterance': 'You will need to get insurance or we will suspend your registration and license'},
  {'da': 'query_condition',
   'references': [{'label': 'precondition', 'sp_id': '28'}],
   'role': 'user',
   'turn_id': 3,
   'utterance': "Don't do that I'll get insurance"},
  {'da': 'respond_solution',
   'references': [{'label': 'solution', 'sp_id': '29'}],
   'role': 'agent',
   'turn_id': 4,
   'utterance': 'Okay, have you received a letter from the DMV letting you

For subtask 1, the gound truths are specified in the reading comprehension dataset 'doc2dial_rc'. Let's show this on the first example from the validation set.

In [ ]:
## Using the doc2dial_rc dataset (also used for evaluation)

qas = load_doc2dial_dataset(
              name="doc2dial_rc",
              split="validation",
            )

Reusing dataset doc2dial (./data_cache/doc2dial/doc2dial_rc/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


In [ ]:
## The following display will be used as ground truth for evaluation
ref = {'id':qas[0]['id'], 'answers':qas[0]['answers']}
ref

{'answers': {'answer_start': [1796],
  'text': ['Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage , ']},
 'id': 'dea7174409afbfe0af0ace21e7f318ae_1'}

The answer corresponding to question `dea7174409afbfe0af0ace21e7f318ae_1` (most likely) has id `dea7174409afbfe0af0ace21e7f318ae_2`. So let's check it using the given dialogue and document domains datasets. We should also check that this answer corresponds to a span starting at `1796` in the associated document.

In [ ]:
## Let's find anwer 'dea7174409afbfe0af0ace21e7f318ae_2'

question_id = 'dea7174409afbfe0af0ace21e7f318ae_2'
dial = 'dea7174409afbfe0af0ace21e7f318ae'
turn = 2

for ex in dialogues:
  if ex['dial_id'] == dial:

    doc_id = ex['doc_id']   ## corresponding document id
    span_ids = []           ## corresponding span ids

    for tr in ex['turns']:
      if tr['turn_id'] == turn:
        print(tr)

        for sp in tr['references']:
          span_ids.append(sp['sp_id'])

{'turn_id': 2, 'role': 'agent', 'da': 'respond_solution', 'references': [{'sp_id': '24', 'label': 'solution'}, {'sp_id': '25', 'label': 'solution'}, {'sp_id': '26', 'label': 'solution'}], 'utterance': 'You will need to get insurance or we will suspend your registration and license'}


In [ ]:
## Let's get the correpond answer in the document_domain

for doc in documents:
  if doc['doc_id'] == doc_id:

    ans = ''
    for span in doc['spans']:
      if span['id_sp'] in span_ids:
        ans += span['text_sp']

print(ans)

Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage , 


In [ ]:
## Let's check that the effectively starts at 1796

for doc in documents:
  if doc['doc_id'] == doc_id:
    for span in doc['spans']:
      if span['start_sp'] == 1796:
        print(span['text_sp'])

Because we all pay indirectly for crashes involving uninsured motorists , 


It checks out! Now that we know that our ground truths are essentially doc2dial_rc, let see what our predictions should look like in order to comply with the evaluation script.

In [ ]:
## Examples of predictions
[
    {
        "id": "dea7174409afbfe0af0ace21e7f318ae_1",
        "prediction_text": "Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage ,",
        "no_answer_probability": 0.883711576461792
    },
    {
        "id": "dea7174409afbfe0af0ace21e7f318ae_9",
        "prediction_text": "we mail you an insurance inquiry letter to allow you to clear up the problem. We send 500,000 inquiry letters a year. ",
        "no_answer_probability": 0.0004379140445962548
    }
]

[{'id': 'dea7174409afbfe0af0ace21e7f318ae_1',
  'no_answer_probability': 0.883711576461792,
  'prediction_text': 'Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage ,'},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_9',
  'no_answer_probability': 0.0004379140445962548,
  'prediction_text': 'we mail you an insurance inquiry letter to allow you to clear up the problem. We send 500,000 inquiry letters a year. '}]